In [118]:
from typing import Literal
# Bib joblib
from joblib import delayed,Parallel
# Numpy
import numpy as np
# Bib skimage 
import skimage
# Bib glob
import glob
# GLCM
from DescritoresDeImagem.glcm import glcm
# LBP
from DescritoresDeImagem.lbp import lbp
#LPQ
from DescritoresDeImagem.lpq import lpq

# Standart Scale, SVC and KNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier


In [119]:
def  glcmimageapply(imgpath,distances=[1,3,5],angles=np.deg2rad([0,90,180,270]))-> np.ndarray[tuple[int],np.dtype[np.float64]]:
    """
    Args
    
    img (np.ndarray): 
        A 2D grayscale image array for which the GLCM is computed. The image 
        should be of type `np.ndarray` and must be 2-dimensional.
            
    distances (Union[List[int], np.ndarray]): 
        A list or array of pixel pair distances for which the GLCM is computed. 
        Default values are [1, 3, 5].
        
    angles (Union[List[float], np.ndarray]): 
        A list or array of angles (in radians) defining the direction for GLCM computation. 
        Default values are the radian equivalents of [0, 90, 180, 270] degrees.
    """
    imgopen=skimage.io.imread(imgpath,as_gray=True) # open image in as gray 
    img_info=glcm(imgopen,distances=distances,angles=angles) # aplly glcm in image open
          
    return img_info 


In [120]:


def lpqimageapply(imgpath, winSize: int = 7,decorr: int = 1,mode: str = 'nh'):
    """
    Applies the LPQ (Local Phase Quantization) method to an image.

    Parameters:
        imgpath (str): Path to the image file.
        winSize (int, optional): Window size for LPQ. Default is 7.
        decorr (int, optional): Decorrelation parameter for LPQ. Default is 1.
        mode (str, optional): Mode for LPQ ('nh' for normalized histogram). Default is 'nh'.

    Returns:
        np.ndarray: Feature vector or matrix produced by the LPQ method.
    """
    imgopen=skimage.io.imread(imgpath,as_gray=True) # open image in as gray 
    img_info=lpq(imgopen,winSize=winSize,mode=mode,decorr=decorr) # aplly glcm in image open
          
    return img_info 


In [121]:
def lbpimageaplly(imgpath,P: int = 8,R: int = 2,method: Literal['default', 'ror', 'uniform', 'nri_uniform', 'var'] = 'nri_uniform'):
    """
    Applies the LBP (Local Binary Patterns) method to an image.

    Parameters:
        imgpath (str): Path to the image file.
        P (int, optional): Number of circularly symmetric neighbor points. Default is 8.
        R (int, optional): Radius of the circle. Default is 2.
        method (Literal, optional): Method to extract LBP. Options include:
            - 'default': Basic LBP.
            - 'ror': Rotation invariant.
            - 'uniform': Uniform patterns only.
            - 'nri_uniform': Non-rotation-invariant uniform patterns.
            - 'var': Variance-based patterns. Default is 'nri_uniform'.

    Returns:
        np.ndarray: The resulting LBP image or feature vector.
    """
    
    imgopen=skimage.io.imread(imgpath,as_gray=True) # open image in as gray 
    
    img_info=lbp(image=imgopen,P=P,R=R,method=method) # aplly glcm in image open
          
    return img_info


In [122]:
def writendarry(filename:str,array:np.ndarray,delimiter="|")->None:
    with open(file=filename,mode="w") as file:
        for i in array:
            file.writelines(np.array2string(i,separator=","))
            file.write(delimiter)
            

In [123]:
def readfilendarray(filename:str,delimiter="|",data="f")->np.ndarray:
    if data=="f":
        array=[]
        with open(file=filename,mode="r") as f:
            lines=f.read().split(delimiter)
            for datastr in lines:
                data=datastr.replace("\n","").replace(" ","").strip("[]").split(",")
                arr=[float(i) for i in data if i.strip()]
                array.append(arr)
            array.pop()
            return np.array(array)
    elif data=="s":
        with open(file=filename,mode="r") as f:
            lines=f.read().replace("'","").split(delimiter)
            lines.pop()
            return np.array(lines)
            

In [124]:
# Number Threads
JOBS=12
# Use descriptor
DESCRIPTOR="LBP" #LPQ,LBP,GLCM
#Run Cell
RUN=True
 
pathimageitetor=glob.glob("BaseDeDados/*.bmp")

if DESCRIPTOR=="LPQ" and RUN: 
    datasetlpq=Parallel(n_jobs=JOBS)(delayed(lbpimageaplly)(i) for i in pathimageitetor)
    writendarry(filename="DadosExtraidos/X_TreinoLPQ.txt",array=datasetlpq)
elif DESCRIPTOR=="LBP" and RUN:
    datasetlbq=Parallel(n_jobs=JOBS)(delayed(lpqimageapply)(i) for i in pathimageitetor)
    writendarry(filename="DadosExtraidos/X_TreinoLBP.txt",array=datasetlbq)
elif DESCRIPTOR=="GLCM" and RUN:
    datasetlbq=Parallel(n_jobs=JOBS)(delayed(glcmimageapply)(i) for i in pathimageitetor)
    writendarry(filename="DadosExtraidos/X_TreinoGLCM.txt",array=datasetlbq)
if RUN:
    y_result=[]
    for i in pathimageitetor:
        y_result.append(i.replace("BaseDeDados/","").replace(".bmp",""))
    writendarry("DadosExtraidos/Y_Resultado.txt",np.array(y_result))


In [ ]:

X_data=readfilendarray(filename="DadosExtraidos/X_TreinoGLCM.txt")

Y_label=readfilendarray(filename="DadosExtraidos/Y_Resultado.txt",data="s")


ValueError: Expected 2D array, got 1D array instead:
array=[2.58285024e+03 1.40140086e+03 2.58285024e+03 1.40140086e+03
 7.45943953e+03 4.19701592e+03 7.45943953e+03 4.19701592e+03
 8.70760761e+03 5.64985821e+03 8.70760761e+03 5.64985821e+03
 1.01288245e+01 5.49568966e+00 1.01288245e+01 5.49568966e+00
 2.92527040e+01 1.64588859e+01 2.92527040e+01 1.64588859e+01
 3.41474808e+01 2.21563067e+01 3.41474808e+01 2.21563067e+01
 9.60279731e-01 9.78448607e-01 9.60279731e-01 9.78448607e-01
 8.85285278e-01 9.35456342e-01 8.85285278e-01 9.35456342e-01
 8.66090370e-01 9.13113859e-01 8.66090370e-01 9.13113859e-01
 8.23220940e-01 8.39796466e-01 8.23220940e-01 8.39796466e-01
 7.57556002e-01 7.97235551e-01 7.57556002e-01 7.97235551e-01
 7.40808098e-01 7.74850616e-01 7.40808098e-01 7.74850616e-01
 9.07315238e-01 9.16404096e-01 9.07315238e-01 9.16404096e-01
 8.70376931e-01 8.92880480e-01 8.70376931e-01 8.92880480e-01
 8.60702096e-01 8.80255995e-01 8.60702096e-01 8.80255995e-01
 7.13487831e-01 8.45081229